## Navegação Assistida por Visão Computacional (T3)

Este notebook implementa o pipeline principal para um sistema de navegação assistida, utilizando:
- **Detecção de Objetos:** YOLOv8n com tracking (ByteTrack).
- **Estimativa de Profundidade:** MiDaS (Small) para profundidade monocular relativa.
- **Processamento Temporal:** Filtro de Média Móvel Simples (SMA) para suavizar a profundidade.
- **Estimativa Métrica:** Conversão da profundidade relativa para metros (requer calibração).
- **Lógica de Navegação:** Geração de comandos direcionais básicos.
- **Feedback Auditivo:** Text-to-Speech (TTS) com rate limiting e detalhes do objeto.

In [1]:
# Bibliotecas Padrão
import time
import os
import json
import tempfile
import threading
from typing import List, Tuple
from collections import deque # Usado no filtro SMA

# Bibliotecas de Terceiros
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from gtts import gTTS

# Configuração para display de áudio
try:
    from IPython.display import Audio, display
    # Tenta importar playsound para reprodução fora do Jupyter
    import playsound
    USE_PLAYSOUND = True
except ImportError:
    USE_PLAYSOUND = False
    print("[Aviso] Biblioteca 'playsound' não encontrada. O áudio será apenas salvo ou tentará usar IPython.display.")
    print("         Para tocar o áudio fora do Jupyter, instale: pip install playsound==1.2.2")

### Carregamento dos Modelos (MiDaS e YOLO)

In [2]:
def load_midas_model(model_type: str = "MiDaS_small"):
    """Carrega o modelo MiDaS especificado e a transformação correspondente."""
    print(f"Carregando modelo MiDaS: {model_type}...")
    # Força recarregar para evitar possíveis problemas de cache do torch.hub
    # torch.hub.download_url_to_file('https://github.com/intel-isl/MiDaS/archive/master.zip', 'midas.zip')
    # !unzip -o midas.zip
    # midas = torch.hub.load("MiDaS-master", model_type, source='local', trust_repo=True) 
    midas = torch.hub.load("intel-isl/MiDaS", model_type, trust_repo=True)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    midas.to(device)
    midas.eval()

    midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms", trust_repo=True)
    # O transform 'small' é compatível com DPT_Hybrid e MiDaS_small
    transform = midas_transforms.small_transform
    print(f"MiDaS carregado no dispositivo: {device}")
    return midas, transform, device

def load_yolo_model(model_name: str = 'yolov8n.pt'):
    """Carrega o modelo YOLO especificado."""
    print(f"Carregando modelo YOLO: {model_name}...")
    try:
        model = YOLO(model_name)
        print("Modelo YOLO carregado com sucesso.")
        # Força uma inferência inicial para otimizar o tempo no primeiro frame
        _ = model(np.zeros((640, 640, 3), dtype=np.uint8))
        print("Modelo YOLO inicializado.")
        return model
    except Exception as e:
        print(f"Erro ao carregar o modelo YOLO: {e}")
        raise e

# Carrega os modelos uma única vez
midas_model, midas_transform, processing_device = load_midas_model("MiDaS_small")
yolo_model = load_yolo_model('yolov8n.pt')

Carregando modelo MiDaS: MiDaS_small...


Using cache found in C:\Users\gabri/.cache\torch\hub\intel-isl_MiDaS_master
c:\Users\gabri\Documents\2025\2sem\MC949_Visao_Computacional\venv-T3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\gabri\Documents\2025\2sem\MC949_Visao_Computacional\venv-T3\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Loading weights:  None


Using cache found in C:\Users\gabri/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master
Using cache found in C:\Users\gabri/.cache\torch\hub\intel-isl_MiDaS_master


MiDaS carregado no dispositivo: cpu
Carregando modelo YOLO: yolov8n.pt...
Modelo YOLO carregado com sucesso.

0: 640x640 (no detections), 507.6ms
Speed: 16.4ms preprocess, 507.6ms inference, 18.0ms postprocess per image at shape (1, 3, 640, 640)
Modelo YOLO inicializado.


### Funções Auxiliares (Processamento e Lógica)

In [3]:
def get_depth(midas, input_batch, img_shape):
    """Executa a inferência do MiDaS e retorna o mapa de profundidade normalizado."""
    with torch.no_grad():
        prediction = midas(input_batch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img_shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()
    output = prediction.cpu().numpy()
    output_min, output_max = output.min(), output.max()
    if output_max > output_min:
        output = (output - output_min) / (output_max - output_min)
    else:
        output = np.zeros_like(output)
    return output

def is_center_in_roi(bbox: tuple[int, int, int, int], roi: tuple[int, int, int, int] | None):
    """Verifica se o centro da BBox está dentro da ROI."""
    if roi is None: return True
    x1, y1, x2, y2 = bbox
    roi_x1, roi_y1, roi_x2, roi_y2 = roi
    obj_center_x = (x1 + x2) / 2
    obj_center_y = (y1 + y2) / 2
    return (roi_x1 <= obj_center_x <= roi_x2) and (roi_y1 <= obj_center_y <= roi_y2)

def get_horizontal_position(bbox: tuple[int, int, int, int], image_width: int):
    """Determina a posição horizontal (Esquerda/Centro/Direita) baseado em terços."""
    x1, _, x2, _ = bbox
    obj_center_x = (x1 + x2) / 2
    limite_esquerda = image_width / 3
    limite_direita = (image_width * 2) / 3
    if obj_center_x < limite_esquerda: return "Esquerda"
    elif obj_center_x <= limite_direita: return "Centro"
    else: return "Direita"

def get_object_depth_dict(yolo_results, depth_frame, img_shape, depth_mode: str = 'median', classes_alvo: list[str] | None = None, roi: tuple[int, int, int, int] | None = None):
    """
    Combina detecções YOLO com mapa de profundidade, calcula profundidade relativa e posição.
    Retorna lista de dicionários por objeto, incluindo 'track_id'.
    """
    if classes_alvo is None:
        classes_alvo = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck', 'bench', 'chair', 'stop sign', 'traffic light']

    MODOS_DE_PROFUNDIDADE_VALIDOS = {'median', 'mean'}
    if depth_mode not in MODOS_DE_PROFUNDIDADE_VALIDOS:
        raise ValueError(f"Modo de profundidade inválido: {depth_mode}")

    detected_objects = []
    nomes_classes = yolo_results.names
    image_height, image_width = img_shape[:2]
    boxes = yolo_results.boxes

    if boxes is None or len(boxes) == 0: return []
    
    # Garante que temos IDs de tracking para associar
    has_track_ids = boxes.id is not None and len(boxes.id) > 0
    track_ids = boxes.id.int().cpu().tolist() if has_track_ids else [-1] * len(boxes)

    for i, box in enumerate(boxes):
        if box.cls is None or len(box.cls) == 0: continue
        classe_id = int(box.cls[0])
        nome = nomes_classes[classe_id]

        if nome in classes_alvo:
            if box.xyxy is None or len(box.xyxy) == 0: continue
            coord_box = box.xyxy[0].int().cpu().numpy()

            if is_center_in_roi(coord_box, roi):
                x1, y1, x2, y2 = coord_box
                y1, y2 = max(0, y1), min(image_height, y2)
                x1, x2 = max(0, x1), min(image_width, x2)
                if y1 >= y2 or x1 >= x2: continue

                profundidades_box = depth_frame[y1:y2, x1:x2]
                if profundidades_box.size == 0: continue

                obj_depth_rel = float(np.median(profundidades_box)) if depth_mode == 'median' else float(np.mean(profundidades_box))
                posicao_horizontal = get_horizontal_position(coord_box, image_width)
                current_track_id = track_ids[i]

                obj_data = {
                    'nome': nome,
                    'profundidade_rel': obj_depth_rel,
                    'bbox': [int(c) for c in coord_box],
                    'posicao': posicao_horizontal,
                    'track_id': current_track_id
                }
                detected_objects.append(obj_data)

    # Não reordena aqui, a ordenação será feita após o filtro temporal
    return detected_objects

def metric_proximity_label(distance_m):
    """Classifica a distância em metros em faixas de proximidade."""
    if distance_m < 1.0: return "Próximo"
    elif distance_m <= 2.5: return "Médio"
    else: return "Longe"

def gerar_comando_navegacao(objetos_filtrados: list) -> Tuple[str, dict | None]:
    """
    Gera comando de navegação baseado no objeto filtrado mais próximo.
    Retorna o comando e os dados do objeto que o gerou (ou None).
    """
    if not objetos_filtrados: return "Siga em frente.", None
    
    obj_mais_proximo = objetos_filtrados[0]
    dist_m = obj_mais_proximo.get('distancia_m', 999)
    pos = obj_mais_proximo['posicao']
    prox_label = metric_proximity_label(dist_m)

    comando = "Siga em frente."
    objeto_causador = None

    if prox_label == "Próximo":
        if pos == "Centro": comando = "Pare!"
        elif pos == "Esquerda": comando = "Desvie à Direita."
        elif pos == "Direita": comando = "Desvie à Esquerda."
        objeto_causador = obj_mais_proximo # Associa o objeto ao comando de alerta
        
    return comando, objeto_causador

### Pipeline Principal: Captura e Processamento em Tempo Real

Integra todas as funcionalidades: captura de vídeo, MiDaS, YOLO com tracking, filtro SMA, conversão métrica, lógica de navegação e TTS com rate limiting.

**Instruções de Uso:**
- Execute a célula abaixo para iniciar a câmera e o processamento
- Para parar o sistema:
  1. **Clique em uma das janelas OpenCV** (`Camera Feed`, `Tracked Feed`, ou `Depth Map`)
  2. **Pressione a tecla 'q'** no seu teclado
- O sistema fornecerá feedback auditivo quando detectar obstáculos próximos

In [ ]:
# Imports para TTS e áudio
from gtts import gTTS
import tempfile
import os
import playsound
import threading # Para tocar áudio sem bloquear o loop principal

# Inicializar a captura de vídeo
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    print("Erro: Não foi possível abrir a câmera.")
    exit()

print("Câmera inicializada. Pressione 'q' para sair.")

# -- Configurações e Variáveis de Estado --
prev_frame_time = time.time()
last_tts_time = 0.0
TTS_RATE_LIMIT_SECONDS = 6.0 # Intervalo mínimo entre TTS
SMA_WINDOW_SIZE = 10
depth_history = {}
last_seen_frame = {}
frame_count = 0
fps_list = deque(maxlen=20)

# -- Constante de Calibração --
# 1. Coloque um objeto a uma distância conhecida (ex: 1.0 metro).
# 2. Rode o código e observe a profundidade RELATIVA FILTRADA (filtered_rel_depth) para o track_id do objeto.
# C = distancia_real_metros * filtered_rel_depth.
CALIBRATION_CONSTANT_C = 0.3 

def relative_to_metric(relative_depth, calibration_constant):
    """Converte profundidade relativa (0-1) para estimativa de distância em metros."""
    return calibration_constant / (relative_depth + 1e-6)

def play_audio_async(file_path):
    """Toca um arquivo de áudio em uma thread separada.
    Se já houver um áudio sendo reproduzido, aguarda até terminar antes de iniciar este.
    Remove o arquivo temporário ao final.
    """
    # Lock para garantir reprodução serial (evita sobreposição de áudios)
    audio_playback_lock = threading.Lock()
    def target():
        try:
            if audio_playback_lock.locked():
                print(f"Áudio já em reprodução. Aguardando para tocar: {file_path}")
            # Espera até conseguir o lock (bloqueia só esta thread, não o loop principal)
            audio_playback_lock.acquire()
            try:
                if USE_PLAYSOUND:
                    try:
                        playsound.playsound(file_path)
                    except Exception as e:
                        print(f"Erro no playsound ou módulo ausente: {e}. Simulando reprodução.")
                        print(f"(Simulando áudio): Tocar {file_path}")
                else:
                    print(f"(Simulando áudio - playsound não disponível): Tocar {file_path}")
            except Exception as e:
                print(f"Erro ao tocar áudio '{file_path}': {e}")
            finally:
                # Pequena pausa antes de remover para garantir flush do player
                time.sleep(0.5)
                try:
                    os.remove(file_path)
                except Exception as e:
                    print(f"Erro ao remover arquivo de áudio temporário '{file_path}': {e}")
                # libera o lock para permitir próxima reprodução
                audio_playback_lock.release()
        except Exception as e:
            print(f"Erro na thread de áudio para '{file_path}': {e}")

    thread = threading.Thread(target=target)
    thread.daemon = True
    thread.start()

# --- Loop Principal ---
try:
    while True:
        ret, frame = cap.read()
        new_frame_time = time.time()
        if not ret: print("Erro: Não foi possível ler o frame."); break
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        input_batch = midas_transform(frame_rgb).to(processing_device)
        output_depth = get_depth(midas_model, input_batch, frame.shape)
        objects_result = yolo_model.track(frame_rgb, persist=True, tracker='bytetrack.yaml', verbose=False)

        altura, largura = frame.shape[:2]
        roi = (largura // 6, 0, (largura * 5) // 6, altura)
        
        # Obtém objetos detectados com profundidade relativa e track_id
        objetos_detectados = get_object_depth_dict(objects_result[0], output_depth, frame.shape, depth_mode='median', roi=roi)

        # -- Filtro Temporal e Conversão Métrica --
        frame_count += 1
        filtered_relative_depths = {}
        metric_depths = {}
        current_track_ids = set()

        for obj in objetos_detectados:
            track_id = obj.get('track_id', -1)
            if track_id != -1:
                current_track_ids.add(track_id)
                last_seen_frame[track_id] = frame_count
                current_rel_depth = obj['profundidade_rel']
                
                history = depth_history.setdefault(track_id, deque(maxlen=SMA_WINDOW_SIZE))
                history.append(current_rel_depth)
                
                filtered_rel_depth = float(np.mean(history))
                filtered_relative_depths[track_id] = filtered_rel_depth
                metric_depths[track_id] = relative_to_metric(filtered_rel_depth, CALIBRATION_CONSTANT_C)

        # -- Limpeza de IDs Antigos --
        if frame_count % 10 == 0:
            ids_to_remove = [tid for tid, last_seen in last_seen_frame.items() if frame_count - last_seen > SMA_WINDOW_SIZE * 3] # Aumentado limiar de limpeza
            for tid in ids_to_remove:
                depth_history.pop(tid, None)
                last_seen_frame.pop(tid, None)

        # -- Preparar Lista Final de Objetos para Navegação --
        objetos_para_navegacao = []
        for obj in objetos_detectados:
            track_id = obj.get('track_id', -1)
            if track_id != -1 and track_id in metric_depths:
                obj_final = obj.copy()
                obj_final['distancia_m'] = metric_depths[track_id]
                obj_final.pop('profundidade_rel', None)
                objetos_para_navegacao.append(obj_final)
            elif track_id == -1: # Fallback para objetos sem tracking
                 obj_final = obj.copy()
                 obj_final['distancia_m'] = relative_to_metric(obj['profundidade_rel'], CALIBRATION_CONSTANT_C)
                 obj_final.pop('profundidade_rel', None)
                 objetos_para_navegacao.append(obj_final)

        objetos_para_navegacao.sort(key=lambda x: x.get('distancia_m', 999))

        # -- Geração de Comando e Feedback --
        comando, objeto_causador = gerar_comando_navegacao(objetos_para_navegacao)
        elapsed_time = new_frame_time - prev_frame_time
        fps_list.append(elapsed_time)
        avg_elapsed_time = np.mean(fps_list)
        fps = 1.0 / avg_elapsed_time if avg_elapsed_time > 0 else 0
        prev_frame_time = new_frame_time
        #print(f"FPS: {fps:.1f} | Comando: {comando}") # Movido para exibição no frame

        # -- Visualização --
        depth_visual = (output_depth * 255).astype(np.uint8)
        depth_colored = cv2.applyColorMap(depth_visual, cv2.COLORMAP_JET)
        frame_tracked_plot = objects_result[0].plot() 
        frame_tracked_bgr = cv2.cvtColor(frame_tracked_plot, cv2.COLOR_RGB2BGR)

        cv2.rectangle(frame, roi[:2], roi[2:], (0, 255, 0), 2)
        cv2.rectangle(frame_tracked_bgr, roi[:2], roi[2:], (0, 255, 0), 2)
        fps_text = f"FPS: {fps:.1f}"
        distancia_text = "--" 
        if objeto_causador: # Usa o objeto que causou o comando
            dist_m = objeto_causador.get('distancia_m', 0)
            distancia_text = f"Dist: {dist_m:.2f}m ({objeto_causador['nome']})"
        elif objetos_para_navegacao: # Se comando é 'Siga', mostra o mais próximo
            dist_m = objetos_para_navegacao[0].get('distancia_m', 0)
            distancia_text = f"Dist: {dist_m:.2f}m ({objetos_para_navegacao[0]['nome']})"
            
        cv2.putText(frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, comando, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, distancia_text, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame_tracked_bgr, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame_tracked_bgr, comando, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(frame_tracked_bgr, distancia_text, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow("Camera Feed", frame)
        cv2.imshow("Tracked Feed", frame_tracked_bgr)
        cv2.imshow("Depth Map", depth_colored)

        # -- Lógica TTS com Rate Limiting e Detalhes --
        current_time = time.time()
        if comando != "Siga em frente." and (current_time - last_tts_time >= TTS_RATE_LIMIT_SECONDS):
            sentence = comando # Começa com o comando principal
            if objeto_causador:
                nome = objeto_causador['nome']
                dist_m = objeto_causador['distancia_m']
                pos = objeto_causador['posicao'].lower()
                # Monta frase mais detalhada para o alerta
                sentence = f"{comando} {nome} a {dist_m:.1f} metros em {pos}."
            
            print(f"TTS Gerado: '{sentence}'") # Log do que será falado
            try:
                tts = gTTS(sentence, lang='pt')
                # Usar um gerenciador de contexto para garantir fechamento
                with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as tmpf:
                    temp_filename = tmpf.name
                tts.save(temp_filename)
                last_tts_time = current_time # Atualiza o timestamp
                play_audio_async(temp_filename) # Toca em thread separada
            except Exception as e:
                print(f"Erro ao gerar ou tocar TTS: {e}")

        # -- Saída --
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    # --- Finalização Segura --- 
    # Garante que a câmera seja liberada e as janelas fechadas mesmo se ocorrer um erro no loop
    cap.release()
    cv2.destroyAllWindows()
    print("Captura de vídeo encerrada e recursos liberados.")


Câmera inicializada. Pressione 'q' para sair.
TTS Gerado: 'Pare! bench a 1.0 metros em centro.'
TTS Gerado: 'Desvie à Esquerda. chair a 0.5 metros em direita.'
TTS Gerado: 'Pare! chair a 1.0 metros em centro.'
TTS Gerado: 'Pare! chair a 0.6 metros em centro.'
TTS Gerado: 'Desvie à Direita. chair a 0.7 metros em esquerda.'
TTS Gerado: 'Desvie à Direita. bench a 0.6 metros em esquerda.'
TTS Gerado: 'Desvie à Esquerda. bench a 1.0 metros em direita.'
TTS Gerado: 'Desvie à Esquerda. chair a 0.5 metros em direita.'
TTS Gerado: 'Pare! chair a 0.5 metros em centro.'
TTS Gerado: 'Pare! chair a 0.6 metros em centro.'
TTS Gerado: 'Desvie à Direita. chair a 0.5 metros em esquerda.'
TTS Gerado: 'Desvie à Esquerda. person a 0.9 metros em direita.'
TTS Gerado: 'Desvie à Direita. bench a 0.9 metros em esquerda.'
TTS Gerado: 'Desvie à Direita. chair a 0.5 metros em esquerda.'
TTS Gerado: 'Desvie à Esquerda. chair a 0.5 metros em direita.'
TTS Gerado: 'Desvie à Direita. bench a 0.8 metros em esquerda.'

KeyboardInterrupt: 

### Funções de Classificação de Proximidade e Geração de Áudio (TTS)

Define as faixas de distância métrica e a lógica para gerar os comandos de voz.

In [5]:
def metric_proximity_label(distance_m):
    """
    Classifica a distância em metros em faixas de proximidade.
    
    Args:
        distance_m (float): Distância estimada em metros.
    
    Returns:
        str: 'Próximo' (< 1m), 'Médio' (1-2.5m), ou 'Longe' (> 2.5m).
    """
    if distance_m < 1.0:
        return "Próximo"
    elif distance_m <= 2.5:
        return "Médio"
    else:
        return "Longe"
